# Solución etapa 3 - FE + Training

In [ ]:
# Utilidades para print
from utils.print_utils import tabl, headr, titl

In [ ]:
# Cargar los datasets
import pandas as pd

historicos_ordenes_cleaned = pd.read_csv('../data/cleaned/historicos_ordenes_cleaned.csv')
registros_condiciones_cleaned = pd.read_csv('../data/cleaned/registros_condiciones_cleaned.csv')
caracteristicas_equipos_cleaned = pd.read_csv('../data/cleaned/caracteristicas_equipos_cleaned.csv')


## Feature eng

### Target

In [ ]:
# Generar target

target_column="Fallo"

def code_target(val):
    if val == 'Correctivo':
        return 1
    else:
        return 0


historicos_ordenes_cleaned[target_column] = historicos_ordenes_cleaned["Tipo_Mantenimiento"].apply(code_target)
historicos_ordenes_cleaned.drop("Tipo_Mantenimiento", axis=1, inplace=True)

### historicos_ordenes

In [ ]:
tabl(historicos_ordenes_cleaned)

In [ ]:
historicos_ordenes_cleaned['Fecha'] = pd.to_datetime(historicos_ordenes_cleaned['Fecha'])

historicos_ordenes_cleaned['Año'] = historicos_ordenes_cleaned['Fecha'].dt.year
historicos_ordenes_cleaned['Mes'] = historicos_ordenes_cleaned['Fecha'].dt.month
historicos_ordenes_cleaned['Día'] = historicos_ordenes_cleaned['Fecha'].dt.day

tabl(historicos_ordenes_cleaned)

In [ ]:
# Tiempo medio entre fallos por fecha

# Filtrar solo los registros con fallos
fallos = historicos_ordenes_cleaned[historicos_ordenes_cleaned['Fallo'] == 1].copy()

# Ordenar por fecha
fallos = fallos.sort_values(by=['ID_Equipo', 'Fecha'])

# Calcular los intervalos entre fallos consecutivos
fallos['Intervalo'] = fallos.groupby('ID_Equipo')['Fecha'].diff()
fallos['Intervalo_Horas'] = fallos['Intervalo'].dt.total_seconds() / 3600

# Calcular el Tiempo medio entre fallos por equipo
mtbf_por_idequipo = fallos.groupby('ID_Equipo')['Intervalo_Horas'].mean().reset_index()
mtbf_por_idequipo.rename(columns={'Intervalo_Horas': 'Tiempo_medio_entre_fallos(h)'}, inplace=True)

tabl(mtbf_por_idequipo)

In [ ]:
# Mezclar historicos_ordenes_cleaned + mtbf_por_idequipo

historicos_mtbf = pd.merge(historicos_ordenes_cleaned, mtbf_por_idequipo, on=['ID_Equipo'], how='left')
tabl(historicos_mtbf)

In [ ]:
# Eliminar columnas sin valor de historicos_mtbf
historicos_mtbf_clean = historicos_mtbf.drop(columns=['ID_Orden','Costo_Mantenimiento','Duracion_Horas','Fecha'])
tabl(historicos_mtbf_clean)

### registros_condiciones

In [ ]:
registros_condiciones_cleaned.drop(columns=['ID_Registro'], index=1, inplace=True)

registros_condiciones_cleaned['Fecha']=pd.to_datetime(registros_condiciones_cleaned['Fecha'])
registros_condiciones_cleaned['Año'] = registros_condiciones_cleaned['Fecha'].dt.year
registros_condiciones_cleaned['Mes'] = registros_condiciones_cleaned['Fecha'].dt.month
registros_condiciones_cleaned['Día'] = registros_condiciones_cleaned['Fecha'].dt.day

tabl(registros_condiciones_cleaned)

In [ ]:
# Agrupar por 'ID_Equipo', 'Año' y 'Mes', y calcular los promedios
promedios_por_equipo_mes = (
    registros_condiciones_cleaned.groupby(['ID_Equipo', 'Año', 'Mes','Día'])[['Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas']]
      .mean()
      .reset_index()
)

# Renombrar las columnas para claridad
promedios_por_equipo_mes.rename(columns={
    'Temperatura_C': 'Promedio_Temperatura_C',
    'Vibracion_mm_s': 'Promedio_Vibracion_mm_s',
    'Horas_Operativas': 'Promedio_Horas_Operativas'
}, inplace=True)

tabl(promedios_por_equipo_mes)

In [ ]:
promedios_por_equipo_mes.isna().sum()
promedios_por_equipo_mes.dropna(inplace=True)
promedios_por_equipo_mes.shape

### Unir merged_hitorico_registro + promedios + caracteristicas

In [ ]:
# Unir datasets
final_data = pd.merge(historicos_mtbf_clean, promedios_por_equipo_mes, on=['ID_Equipo','Año','Mes'], how='inner')
final_data = pd.merge(final_data, caracteristicas_equipos_cleaned, on='ID_Equipo', how='inner')

tabl(final_data)

In [ ]:
# Limpiar columnas accesorias
final_data.drop(columns=['ID_Equipo','Año', 'Mes', 'Día_x', 'Día_y'], inplace=True)

tabl(final_data)

### Duplicados

In [ ]:
# Eliminar duplicados

print(headr('Duplicados final_data:'),final_data.duplicated().sum())
# final_data.drop_duplicates(inplace=True)
# tabl(final_data)

In [ ]:
# Verificamos nulos
print(headr('Nulos final_data:'),final_data.isna().sum())

In [ ]:
columnas_con_nulos = final_data.columns[final_data.isna().any()].tolist()
columnas_con_nulos

In [ ]:
# Imputamos nulos
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

# final_data[columnas_con_nulos] = imputer.fit_transform(final_data[columnas_con_nulos])

In [ ]:
tabl(final_data)

In [ ]:
final_data.info()

### Volvemos a explorar con columnas codificadas


In [ ]:
# Extraer columnas numéricas y categóricas
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(final_data)
categorical_columns = categorical_columns_selector(final_data)

print(headr("Numerical columns"), numerical_columns)
print(headr("Categorical columns"), categorical_columns)

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

final_data_coded =final_data.copy()
final_data_coded[categorical_columns] =final_data_coded[categorical_columns].apply(LabelEncoder().fit_transform)   
tabl(final_data_coded)

In [ ]:
# Evaluamos la aportación de cada columna
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif,chi2

X= final_data_coded.drop(target_column, axis=1)
y = final_data_coded[target_column]

fvalue_selector = SelectKBest(f_classif, k=2)

X_kbest = fvalue_selector.fit(X,y)

feature_scores = pd.DataFrame({"Feature": X.columns,"Score": X_kbest.scores_}).sort_values(by="Score", ascending=False)

print(headr("Feature scores"))
round(feature_scores,2)

> Una característica tiene demasiado peso y determina toda la clasificación

In [ ]:
# Eliminamos columnas sin peso

columns_to_drop = feature_scores[feature_scores['Score'] == 0]['Feature']
final_data.drop(columns=columns_to_drop, inplace=True)
tabl(final_data)

## Exportar 

In [ ]:
final_data.to_csv('../data/preprocessed/preprocessed_data_v1.csv', index=False)